In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import os
import random
import warnings
from glob import glob
import ipywidgets as widgets
%matplotlib inline

from lib import *

import numpy as np
import pandas as pd

from itertools import chain
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.morphology import label
from skimage.io import imread, imshow, imread_collection, concatenate_images

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf

# Set some parameters
IMG_WIDTH = 240
IMG_HEIGHT = 320
IMG_CHANNELS = 3

seed = 42
random.seed = seed
np.random.seed = seed

warnings.filterwarnings('ignore')

c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

# KERAS model of U-net and learning

In [2]:
# Build U-Net model
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)
model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 240, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 320, 240, 3)  0           input_1[0][0]                    
_______________________________________________________________________________

conv2d_14 (Conv2D)              (None, 80, 60, 64)   36928       dropout_7[0][0]                  
__________________________________________________________________________________________________
conv2d_transpose_3 (Conv2DTrans (None, 160, 120, 32) 8224        conv2d_14[0][0]                  
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 160, 120, 64) 0           conv2d_transpose_3[0][0]         
                                                                 conv2d_4[0][0]                   
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 160, 120, 32) 18464       concatenate_3[0][0]              
__________________________________________________________________________________________________
dropout_8 (Dropout)             (None, 160, 120, 32) 0           conv2d_15[0][0]                  
__________

In [3]:
# Get train imgs and masks
X_train = Get_IMGs('data\\train\\') 
Y_train = Get_Masks('data\\train_mask\\')

# Fit model
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('model_1.h5', verbose=0, save_best_only=True)
results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=16, epochs=50, 
                    callbacks=[checkpointer,earlystopper], verbose=0)

100%|█████████████████████████████████████████████████████████████████████████████| 1315/1315 [00:06<00:00, 208.86it/s]


Train on 1183 samples, validate on 132 samples
Epoch 1/50
1183/1183 [==============================] - ETA: 5:15 - loss: 0.7563 - mean_iou: 0.0000e+ - ETA: 2:43 - loss: 0.7408 - mean_iou: 0.1411   - ETA: 1:53 - loss: 0.8979 - mean_iou: 0.18 - ETA: 1:28 - loss: 0.8396 - mean_iou: 0.21 - ETA: 1:13 - loss: 0.8140 - mean_iou: 0.23 - ETA: 1:03 - loss: 0.7944 - mean_iou: 0.25 - ETA: 55s - loss: 0.7809 - mean_iou: 0.2579 - ETA: 50s - loss: 0.7699 - mean_iou: 0.262 - ETA: 45s - loss: 0.7613 - mean_iou: 0.266 - ETA: 41s - loss: 0.7512 - mean_iou: 0.269 - ETA: 38s - loss: 0.7435 - mean_iou: 0.270 - ETA: 36s - loss: 0.7374 - mean_iou: 0.271 - ETA: 33s - loss: 0.7319 - mean_iou: 0.272 - ETA: 32s - loss: 0.7275 - mean_iou: 0.273 - ETA: 30s - loss: 0.7216 - mean_iou: 0.274 - ETA: 28s - loss: 0.7162 - mean_iou: 0.275 - ETA: 27s - loss: 0.7149 - mean_iou: 0.276 - ETA: 26s - loss: 0.7103 - mean_iou: 0.277 - ETA: 25s - loss: 0.7071 - mean_iou: 0.277 - ETA: 24s - loss: 0.7051 - mean_iou: 0.278 - ETA: 23s

1183/1183 [==============================] - ETA: 18s - loss: 0.4584 - mean_iou: 0.433 - ETA: 18s - loss: 0.4805 - mean_iou: 0.434 - ETA: 17s - loss: 0.4787 - mean_iou: 0.434 - ETA: 17s - loss: 0.4740 - mean_iou: 0.434 - ETA: 17s - loss: 0.4609 - mean_iou: 0.434 - ETA: 16s - loss: 0.4622 - mean_iou: 0.435 - ETA: 16s - loss: 0.4800 - mean_iou: 0.435 - ETA: 16s - loss: 0.4761 - mean_iou: 0.435 - ETA: 15s - loss: 0.4705 - mean_iou: 0.436 - ETA: 15s - loss: 0.4681 - mean_iou: 0.436 - ETA: 15s - loss: 0.4676 - mean_iou: 0.436 - ETA: 15s - loss: 0.4709 - mean_iou: 0.436 - ETA: 15s - loss: 0.4693 - mean_iou: 0.437 - ETA: 14s - loss: 0.4692 - mean_iou: 0.437 - ETA: 14s - loss: 0.4661 - mean_iou: 0.437 - ETA: 14s - loss: 0.4676 - mean_iou: 0.437 - ETA: 14s - loss: 0.4643 - mean_iou: 0.438 - ETA: 13s - loss: 0.4629 - mean_iou: 0.438 - ETA: 13s - loss: 0.4624 - mean_iou: 0.438 - ETA: 13s - loss: 0.4614 - mean_iou: 0.438 - ETA: 12s - loss: 0.4608 - mean_iou: 0.439 - ETA: 12s - loss: 0.4577 - mean_

1183/1183 [==============================] - ETA: 19s - loss: 0.4878 - mean_iou: 0.505 - ETA: 18s - loss: 0.4174 - mean_iou: 0.505 - ETA: 17s - loss: 0.4182 - mean_iou: 0.505 - ETA: 17s - loss: 0.4104 - mean_iou: 0.506 - ETA: 17s - loss: 0.4060 - mean_iou: 0.506 - ETA: 16s - loss: 0.4083 - mean_iou: 0.506 - ETA: 16s - loss: 0.4014 - mean_iou: 0.506 - ETA: 16s - loss: 0.3990 - mean_iou: 0.506 - ETA: 16s - loss: 0.3927 - mean_iou: 0.506 - ETA: 15s - loss: 0.3909 - mean_iou: 0.506 - ETA: 15s - loss: 0.3948 - mean_iou: 0.507 - ETA: 15s - loss: 0.3928 - mean_iou: 0.507 - ETA: 15s - loss: 0.3913 - mean_iou: 0.507 - ETA: 14s - loss: 0.3928 - mean_iou: 0.507 - ETA: 14s - loss: 0.3912 - mean_iou: 0.507 - ETA: 14s - loss: 0.3900 - mean_iou: 0.507 - ETA: 14s - loss: 0.3920 - mean_iou: 0.507 - ETA: 13s - loss: 0.3945 - mean_iou: 0.508 - ETA: 13s - loss: 0.3902 - mean_iou: 0.508 - ETA: 13s - loss: 0.3947 - mean_iou: 0.508 - ETA: 13s - loss: 0.3927 - mean_iou: 0.508 - ETA: 12s - loss: 0.3901 - mean_

1183/1183 [==============================] - ETA: 16s - loss: 0.3541 - mean_iou: 0.545 - ETA: 17s - loss: 0.3581 - mean_iou: 0.545 - ETA: 16s - loss: 0.3343 - mean_iou: 0.545 - ETA: 16s - loss: 0.3353 - mean_iou: 0.546 - ETA: 16s - loss: 0.3603 - mean_iou: 0.546 - ETA: 16s - loss: 0.3517 - mean_iou: 0.546 - ETA: 15s - loss: 0.3547 - mean_iou: 0.546 - ETA: 15s - loss: 0.3575 - mean_iou: 0.546 - ETA: 15s - loss: 0.3619 - mean_iou: 0.546 - ETA: 15s - loss: 0.3629 - mean_iou: 0.546 - ETA: 15s - loss: 0.3640 - mean_iou: 0.546 - ETA: 14s - loss: 0.3613 - mean_iou: 0.546 - ETA: 14s - loss: 0.3588 - mean_iou: 0.546 - ETA: 14s - loss: 0.3561 - mean_iou: 0.547 - ETA: 14s - loss: 0.3546 - mean_iou: 0.547 - ETA: 14s - loss: 0.3544 - mean_iou: 0.547 - ETA: 13s - loss: 0.3504 - mean_iou: 0.547 - ETA: 13s - loss: 0.3481 - mean_iou: 0.547 - ETA: 13s - loss: 0.3444 - mean_iou: 0.547 - ETA: 13s - loss: 0.3484 - mean_iou: 0.547 - ETA: 12s - loss: 0.3485 - mean_iou: 0.547 - ETA: 12s - loss: 0.3468 - mean_

...

1183/1183 [==============================] - ETA: 18s - loss: 0.1855 - mean_iou: 0.645 - ETA: 18s - loss: 0.1917 - mean_iou: 0.645 - ETA: 18s - loss: 0.1965 - mean_iou: 0.646 - ETA: 17s - loss: 0.1982 - mean_iou: 0.646 - ETA: 17s - loss: 0.2036 - mean_iou: 0.646 - ETA: 16s - loss: 0.1968 - mean_iou: 0.646 - ETA: 16s - loss: 0.1948 - mean_iou: 0.646 - ETA: 16s - loss: 0.1921 - mean_iou: 0.646 - ETA: 16s - loss: 0.1900 - mean_iou: 0.646 - ETA: 16s - loss: 0.1966 - mean_iou: 0.646 - ETA: 15s - loss: 0.1945 - mean_iou: 0.646 - ETA: 15s - loss: 0.1931 - mean_iou: 0.646 - ETA: 15s - loss: 0.1905 - mean_iou: 0.646 - ETA: 15s - loss: 0.1916 - mean_iou: 0.646 - ETA: 14s - loss: 0.1924 - mean_iou: 0.646 - ETA: 14s - loss: 0.1952 - mean_iou: 0.646 - ETA: 14s - loss: 0.1947 - mean_iou: 0.646 - ETA: 13s - loss: 0.1936 - mean_iou: 0.646 - ETA: 13s - loss: 0.1951 - mean_iou: 0.647 - ETA: 13s - loss: 0.1968 - mean_iou: 0.647 - ETA: 12s - loss: 0.1965 - mean_iou: 0.647 - ETA: 12s - loss: 0.1978 - mean_

1183/1183 [==============================] - ETA: 18s - loss: 0.1987 - mean_iou: 0.664 - ETA: 17s - loss: 0.1616 - mean_iou: 0.665 - ETA: 17s - loss: 0.1768 - mean_iou: 0.665 - ETA: 16s - loss: 0.1798 - mean_iou: 0.665 - ETA: 16s - loss: 0.1838 - mean_iou: 0.665 - ETA: 16s - loss: 0.1792 - mean_iou: 0.665 - ETA: 16s - loss: 0.1763 - mean_iou: 0.665 - ETA: 15s - loss: 0.1722 - mean_iou: 0.665 - ETA: 15s - loss: 0.1704 - mean_iou: 0.665 - ETA: 15s - loss: 0.1730 - mean_iou: 0.665 - ETA: 15s - loss: 0.1712 - mean_iou: 0.665 - ETA: 14s - loss: 0.1674 - mean_iou: 0.665 - ETA: 14s - loss: 0.1677 - mean_iou: 0.665 - ETA: 14s - loss: 0.1703 - mean_iou: 0.665 - ETA: 14s - loss: 0.1715 - mean_iou: 0.665 - ETA: 13s - loss: 0.1700 - mean_iou: 0.665 - ETA: 13s - loss: 0.1730 - mean_iou: 0.665 - ETA: 13s - loss: 0.1719 - mean_iou: 0.665 - ETA: 13s - loss: 0.1712 - mean_iou: 0.665 - ETA: 12s - loss: 0.1708 - mean_iou: 0.666 - ETA: 12s - loss: 0.1689 - mean_iou: 0.666 - ETA: 12s - loss: 0.1678 - mean_

1183/1183 [==============================] - ETA: 18s - loss: 0.1815 - mean_iou: 0.681 - ETA: 17s - loss: 0.1501 - mean_iou: 0.681 - ETA: 17s - loss: 0.1510 - mean_iou: 0.681 - ETA: 17s - loss: 0.1580 - mean_iou: 0.681 - ETA: 17s - loss: 0.1620 - mean_iou: 0.681 - ETA: 16s - loss: 0.1586 - mean_iou: 0.681 - ETA: 16s - loss: 0.1660 - mean_iou: 0.681 - ETA: 16s - loss: 0.1735 - mean_iou: 0.681 - ETA: 16s - loss: 0.1745 - mean_iou: 0.681 - ETA: 15s - loss: 0.1724 - mean_iou: 0.681 - ETA: 15s - loss: 0.1766 - mean_iou: 0.681 - ETA: 15s - loss: 0.1778 - mean_iou: 0.682 - ETA: 15s - loss: 0.1756 - mean_iou: 0.682 - ETA: 15s - loss: 0.1732 - mean_iou: 0.682 - ETA: 14s - loss: 0.1725 - mean_iou: 0.682 - ETA: 14s - loss: 0.1726 - mean_iou: 0.682 - ETA: 14s - loss: 0.1768 - mean_iou: 0.682 - ETA: 14s - loss: 0.1751 - mean_iou: 0.682 - ETA: 13s - loss: 0.1743 - mean_iou: 0.682 - ETA: 13s - loss: 0.1730 - mean_iou: 0.682 - ETA: 13s - loss: 0.1758 - mean_iou: 0.682 - ETA: 12s - loss: 0.1724 - mean_

1183/1183 [==============================] - ETA: 18s - loss: 0.1015 - mean_iou: 0.696 - ETA: 17s - loss: 0.1105 - mean_iou: 0.696 - ETA: 17s - loss: 0.1189 - mean_iou: 0.696 - ETA: 17s - loss: 0.1223 - mean_iou: 0.696 - ETA: 17s - loss: 0.1239 - mean_iou: 0.696 - ETA: 17s - loss: 0.1261 - mean_iou: 0.696 - ETA: 16s - loss: 0.1249 - mean_iou: 0.696 - ETA: 16s - loss: 0.1278 - mean_iou: 0.696 - ETA: 16s - loss: 0.1289 - mean_iou: 0.696 - ETA: 16s - loss: 0.1283 - mean_iou: 0.696 - ETA: 15s - loss: 0.1271 - mean_iou: 0.696 - ETA: 15s - loss: 0.1277 - mean_iou: 0.696 - ETA: 15s - loss: 0.1252 - mean_iou: 0.697 - ETA: 14s - loss: 0.1267 - mean_iou: 0.697 - ETA: 14s - loss: 0.1277 - mean_iou: 0.697 - ETA: 14s - loss: 0.1285 - mean_iou: 0.697 - ETA: 14s - loss: 0.1287 - mean_iou: 0.697 - ETA: 13s - loss: 0.1315 - mean_iou: 0.697 - ETA: 13s - loss: 0.1306 - mean_iou: 0.697 - ETA: 13s - loss: 0.1291 - mean_iou: 0.697 - ETA: 13s - loss: 0.1297 - mean_iou: 0.697 - ETA: 12s - loss: 0.1294 - mean_

1183/1183 [==============================] - ETA: 19s - loss: 0.0931 - mean_iou: 0.710 - ETA: 17s - loss: 0.0981 - mean_iou: 0.710 - ETA: 16s - loss: 0.0957 - mean_iou: 0.710 - ETA: 16s - loss: 0.1030 - mean_iou: 0.710 - ETA: 16s - loss: 0.1015 - mean_iou: 0.710 - ETA: 16s - loss: 0.1076 - mean_iou: 0.711 - ETA: 16s - loss: 0.1076 - mean_iou: 0.711 - ETA: 16s - loss: 0.1070 - mean_iou: 0.711 - ETA: 15s - loss: 0.1068 - mean_iou: 0.711 - ETA: 15s - loss: 0.1063 - mean_iou: 0.711 - ETA: 15s - loss: 0.1056 - mean_iou: 0.711 - ETA: 14s - loss: 0.1042 - mean_iou: 0.711 - ETA: 14s - loss: 0.1068 - mean_iou: 0.711 - ETA: 14s - loss: 0.1068 - mean_iou: 0.711 - ETA: 14s - loss: 0.1069 - mean_iou: 0.711 - ETA: 13s - loss: 0.1068 - mean_iou: 0.711 - ETA: 13s - loss: 0.1059 - mean_iou: 0.711 - ETA: 13s - loss: 0.1065 - mean_iou: 0.711 - ETA: 13s - loss: 0.1066 - mean_iou: 0.711 - ETA: 13s - loss: 0.1071 - mean_iou: 0.711 - ETA: 12s - loss: 0.1082 - mean_iou: 0.711 - ETA: 12s - loss: 0.1081 - mean_

# Results

Test model on validation images, save to csv and get Dice metric

In [4]:
model = load_model('model_1.h5', custom_objects={'mean_iou': mean_iou})
valid_imgs = Get_IMGs('data\\valid\\') #get np.array with images
valid_true_masks = Get_Masks('data\\valid_mask\\')#get np.array with masks

valid_pred_masks = (model.predict(valid_imgs)> 0.5).astype(np.uint8) #get predicted masks

# Save valid rle_masks to pred_valid_template.csv
df = pd.DataFrame({
        'id': [int(x.split('.')[0]) for x in next(os.walk('data\\valid\\'))[2]],
        'rle_mask': [encode_rle(mask) for mask in np.squeeze(valid_pred_masks)]
})
df.to_csv('data/pred_valid_template.csv',index=False)  #save to csv

#Check Dice metric
Dice_metric = get_dice(np.squeeze(valid_true_masks), np.squeeze(valid_pred_masks))
print('Dice metric for valid imgs:', Dice_metric)

100%|███████████████████████████████████████████████████████████████████████████████| 145/145 [00:00<00:00, 215.11it/s]


Dice metric for valid imgs: 0.9338391916772382


# Dice metric for valid imgs: 0.9338

Show image, true mask and predicted mask

In [5]:
@interact(i=(0,len(valid_imgs),1))
def g(i=0):
    plt.figure(figsize=(10,4))
    
    plt.subplot(1,3,1)
    plt.title('image')
    plt.imshow(valid_imgs[i,:,:])
    
    plt.subplot(1,3,2)
    plt.title('true mask')
    plt.imshow(valid_true_masks[i,:,:,0])
    
    plt.subplot(1,3,3)
    plt.title('mask predicted')
    plt.imshow(valid_pred_masks[i,:,:,0])

interactive(children=(IntSlider(value=0, description='i', max=145), Output()), _dom_classes=('widget-interact'…

Get masks from test images and create examples.html 

In [6]:
test_imgs = Get_IMGs('data/test/') 

# Predict masks for test imgs
model = load_model('model_1.h5', custom_objects={'mean_iou': mean_iou})
test_pred_masks = model.predict(test_imgs)
test_pred_masks = np.squeeze(test_pred_masks > 0.5).astype(np.uint8)*255 
# without *255 masks, putted in example.html, will look like black rectangle

# Save to html
paths_to_imgs = sorted(glob("data\\test\\*"))
_ = get_html(paths_to_imgs, test_pred_masks, path_to_save="results\\example")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 74.56it/s]
